# Intro to Poke: Polarization Ray Tracing

The first physics submodule of poke is for polarization ray tracing (PRT). All of the physics are done in the `poke.polarization` module, and everything else is just ray data. PRT is an expression of the Fresnel Equations (citation needed) for thin-film polarization in three dimensions. This allows for the propagation of polarization-dependent performance through a ray trace of an optical system

The desireable data product is a Jones Pupil, which is the 3x3 PRT matrix rotated into a local coordinate system. Poke does this using the double pole coordinate system descibed in Chipman, Lam, and Young (2018) Chapter 11.4. This coordinate system is robust to polarization singularities that arise in using the s- and p- basis.

## Initializing a Rayfront as Polarized
So you want a Jones pupil of an optical system, this section will describe how we set up optical system parameters for a `Rayfront` to be built. First, we start with the system properties:
- path to Zemax sequential ray trace file
- thin film information

In [1]:
n_film = 0.04 - 1j*7.1155 # Ag at 1um from Johnson and Christy 1972 refractiveindex.info
pth_to_zmx = 'C:/Users/douglase/Desktop/poke/test_files/Hubble_Test.zmx'

That wasn't too bad. Note that we only specify a thin film iindex, which means that the software assumes the substrate is made of entirely silver. poke also supports thin film stacks, but we will cover that in another tutorial. Now we must specify the surface data. poke handles surface data with dictionaries as a low-level "user interface", and stores them in a list in the order that they appear in the raytrace.

In [2]:
# The Primary Mirror
s1 = {
    "surf" : 1,         # surface number in zemax
    "coating" : n_film, # refractive index of surface
    "mode" : "reflect"  # compute in reflection ("reflect") or transmission ("transmit")
}

# The Secondary Mirror
s2 = {
    "surf" : 2,
    "coating" : n_film,
    "mode" : "reflect"
}

surflist = [s1,s2]

Now that we have the surface information, we can initialize a `Rayfront` and pass this surface data to it.

In [3]:
from poke.poke_core import Rayfront

# rayfront parameters
number_of_rays = 11 # across the entrance pupil
wavelength = 1e-6
pupil_radius = 1.2 # semi-aperture of Hubble
max_field_of_view = 0.08 # degrees

rays = Rayfront(number_of_rays,wavelength,pupil_radius,max_field_of_view)

norm fov =  [0. 0.]
base ray shape  (4, 69)


Now we have a standard Rayfront, which is just a bundle of un-traced rays now. To enable the physical optics capabilities, we need to call the `Rayfront.as_polarized()` method and pass it the surface list.

In [4]:
rays.as_polarized(surflist)

Then we can propagate it through the optical system with the `rays.TraceRaysZOS()` method by supplying the path specified earlier

In [5]:
rays.TraceRaysetZOS(pth_to_zmx)

FileNotFoundException: Could not load file or assembly 'ZOSAPI_Interfaces, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null' or one of its dependencies. The system cannot find the file specified.
File name: 'ZOSAPI_Interfaces, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null'
   at System.Signature.GetSignature(Void* pCorSig, Int32 cCorSig, RuntimeFieldHandleInternal fieldHandle, IRuntimeMethodInfo methodHandle, RuntimeType declaringType)
   at System.Reflection.RuntimeConstructorInfo.GetParametersNoCopy()
   at System.Reflection.MethodBase.GetParameterTypes()
   at System.Reflection.MethodBase.FormatNameAndSig(Boolean serialization)
   at System.Reflection.RuntimeConstructorInfo.ToString()
   at Python.Runtime.MethodObject..ctor(MaybeType type, String name, MethodBase[] info, Boolean allow_threads)
   at Python.Runtime.ClassManager.GetClassInfo(Type type, ClassBase impl)
   at Python.Runtime.ClassManager.InitClassBase(Type type, ClassBase impl, ReflectedClrType pyTy

Now the rays have position and angle

In [8]:
! pip list

Package                Version     Editable project location
---------------------- ----------- ------------------------------
anyio                  3.5.0
argon2-cffi            21.3.0
argon2-cffi-bindings   21.2.0
asdf                   2.14.3
asdf-standard          1.0.3
asdf-transform-schemas 0.3.0
asdf-unit-schemas      0.1.0
astropy                5.1.1
asttokens              2.2.1
atomicwrites           1.4.1
attrs                  22.1.0
Babel                  2.9.1
backcall               0.2.0
beautifulsoup4         4.11.1
bleach                 4.1.0
brotlipy               0.7.0
certifi                2022.9.24
cffi                   1.15.1
charset-normalizer     2.0.4
clr-loader             0.2.4
colorama               0.4.6
contourpy              1.0.6
coverage               7.0.4
cryptography           38.0.1
cycler                 0.11.0
debugpy                1.5.1
decorator              5.1.1
defusedxml             0.7.1
distlib                0.3.6
entrypoints         

In [10]:
import zosapi, os
zos = zosapi.App()
zos.TheSystem.LoadFile(os.path.join(os.path.sep, zos.TheApplication.SamplesDir, r'Sequential\Objectives\Double Gauss 28 degree field.zmx'), False)
if zos.TheSystem.SystemData.Aperture.ApertureValue == 0:
    raise SystemError('Did not properly load a file')
    print('no error found')

In [23]:
psf = zos.TheSystem.Analyses.New_FftPsf()

TypeError: No method matches given arguments for IAR_.GetDataGrid: ()